In [8]:
import sys
sys.path.append('..')
import CAT
import json
import logging
import numpy as np
import pandas as pd

In [9]:
def setuplogger():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(levelname)s %(asctime)s] %(message)s")
    handler.setFormatter(formatter)
    root.addHandler(handler)

In [10]:
setuplogger()

In [11]:
# choose dataset here
dataset = 'assistment'
# modify config here
config = {
    'learning_rate': 0.002,
    'batch_size': 2048,
    'num_epochs': 10,
    'num_dim': 10, # for IRT or MIRT
    'device': 'cpu',
    # for NeuralCD
    'prednet_len1': 128,
    'prednet_len2': 64,
    'betas':(0.9, 0.999),
}

In [14]:
# read datasets
train_triplets = pd.read_csv(f'./dataset/train_triples.csv', encoding='utf-8').to_records(index=False)
concept_map = json.load(open(f'./dataset/concept_map.json', 'r'))
concept_map = {int(k):v for k,v in concept_map.items()}
metadata = json.load(open(f'./dataset/metadata.json', 'r'))

In [15]:
train_data = CAT.dataset.TrainDataset(train_triplets, concept_map,
                                      metadata['num_train_students'], 
                                      metadata['num_questions'], 
                                      metadata['num_concepts'])

In [17]:
# define model here
model = CAT.model.IRTModel(**config)
# train model
model.init_model(train_data)
model.train(train_data, log_step=10)

[INFO 2024-06-08 17:47:52,872] train on cpu
[INFO 2024-06-08 17:47:52,872] train on cpu
[INFO 2024-06-08 17:47:52,933] Epoch [1] Batch [0]: loss=inf
[INFO 2024-06-08 17:47:52,933] Epoch [1] Batch [0]: loss=inf
[INFO 2024-06-08 17:47:53,584] Epoch [1] Batch [10]: loss=0.76193
[INFO 2024-06-08 17:47:53,584] Epoch [1] Batch [10]: loss=0.76193
[INFO 2024-06-08 17:47:54,126] Epoch [1] Batch [20]: loss=0.72686
[INFO 2024-06-08 17:47:54,126] Epoch [1] Batch [20]: loss=0.72686
[INFO 2024-06-08 17:47:54,679] Epoch [1] Batch [30]: loss=0.71491
[INFO 2024-06-08 17:47:54,679] Epoch [1] Batch [30]: loss=0.71491
[INFO 2024-06-08 17:47:55,232] Epoch [1] Batch [40]: loss=0.70873
[INFO 2024-06-08 17:47:55,232] Epoch [1] Batch [40]: loss=0.70873
[INFO 2024-06-08 17:47:55,802] Epoch [1] Batch [50]: loss=0.70485
[INFO 2024-06-08 17:47:55,802] Epoch [1] Batch [50]: loss=0.70485
[INFO 2024-06-08 17:47:56,359] Epoch [1] Batch [60]: loss=0.70213
[INFO 2024-06-08 17:47:56,359] Epoch [1] Batch [60]: loss=0.7021

In [18]:
# save model
model.adaptest_save('../ckpt/mirt.pt')